In [ ]:
def standardize(data):
    '''对数据做标准化处理'''
    mu = np.mean(data, axis=0)
    sigma = np.std(data, axis=0)
    return (data - mu) / sigma

和画图有关的函数：

In [ ]:
from matplotlib import pyplot as plt
from matplotlib import font_manager

# 按需下载字体至本地文件夹目录
simsun=font_manager.FontProperties(fname="C:\Windows\Fonts\simsun.ttc")
timesNewRoman=font_manager.FontProperties(fname="C:\Windows\Fonts/times.ttc")
simtime=font_manager.FontProperties(fname="C:/Users/10503/AppData/Local/Microsoft/Windows/Fonts/tnw+simsun.ttf")

def plot_frames(frames, interval=1, add_colorbar=False, mirror_left=False, mirror_right=False, axis_off=False):    
    '''给定一个结构的拓扑优化的单元密度迭代历史，画出其每一步的密度图。'''
    frames = np.array(frames)
    if len(frames.shape) == 2: # 如果输入数据只有两个维度，说明只有单个迭代步的图，只画这一张
      if axis_off:
        plt.xticks([])
        plt.yticks([])
      plt.imshow(mirror_frame(frames, mirror_left, mirror_right), cmap='Greys')
      if add_colorbar:
          plt.colorbar()
      plt.show()
      return
    frames = mirror_frames(frames, mirror_left, mirror_right)
    key_frames = len(frames)//interval
    columns = 5
    rows = key_frames//columns + 1
    fig = plt.figure(figsize=(20, 20))
    for i in range(1, key_frames + 1):
        step = i * interval - 1
        fig.add_subplot(rows, columns, i)
        plt.title(f'step{step}')
        plt.imshow(frames[step], cmap='Greys')
        plt.axis('off')
        if add_colorbar:
          plt.colorbar()
    plt.show()

def mirror_frames(frames, mirror_left, mirror_right):
  '''如果结构是对称的（取了1/2或者1/3结构作为拓扑优化的输入），则在本函数中对其进行还原'''
  mirror = np.flip(frames, 2)
  if not mirror_left and not mirror_right:
    return frames
  elif mirror_left and mirror_right:
    new_frames = np.concatenate((mirror,frames,mirror),axis=2)
  elif mirror_left:
    new_frames = np.concatenate((mirror,frames), axis=2)
  elif mirror_right:
    new_frames = np.concatenate((frames,mirror), axis=2)
  return new_frames

def mirror_frame(frame, mirror_left, mirror_right):
  mirror = np.flip(frame, 1)
  if not mirror_left and not mirror_right:
    return frame
  elif mirror_left and mirror_right:
    new_frame = np.concatenate((mirror,frame,mirror),axis=1)
  elif mirror_left:
    new_frame = np.concatenate((mirror,frame), axis=1)
  elif mirror_right:
    new_frame = np.concatenate((frame,mirror), axis=1)
  return new_frame

def plot_sens(frames,interval=1, add_colorbar=False,axis_off=False):  
    '''给定一个结构的拓扑优化的单元灵敏度迭代历史，画出其每一步的灵敏度图。'''  
    frames = np.array(frames)
    if len(frames.shape) == 2:
      plt.imshow(frames, cmap='viridis')
      if axis_off:
        plt.xticks([])
        plt.yticks([])
      if add_colorbar:
          plt.colorbar(fontproperties=simsun)
      plt.show()
      return
    key_frames = len(frames)//interval
    columns = 5
    rows = key_frames//columns + 1
    fig = plt.figure(figsize=(20, 20))
    for i in range(1, key_frames + 1):
        step = i * interval - 1
        fig.add_subplot(rows, columns, i)
        plt.title(f'step{step}')
        plt.imshow(frames[step], cmap='viridis')
        plt.axis('off')
        if add_colorbar:
          plt.colorbar()
    plt.show()

def plot_losses(losses):
    '''画出一个结构的拓扑优化的目标函数下降历史'''  
    fig,axes = plt.subplots(figsize=(5,5))
    step = np.arange(len(losses))
    axes.plot(step, losses)
    # 2.2 axes对象添加图名称，坐标轴名称
    axes.set_xlabel("迭代步数",fontproperties=simsun, fontsize=15)
    axes.set_ylabel("目标函数",fontproperties=simsun, fontsize=15)
    axes.set_title("拓扑优化目标函数的下降历史",fontproperties=simsun, fontsize=15)
    # axes.legend() # 添加图例

     # 2.3 坐标轴刻度字体设置
    x1_label = axes.get_xticklabels() 
    [x1_label_temp.set_fontname('Times New Roman') for x1_label_temp in x1_label]
    [x1_label_temp.set_fontsize(12) for x1_label_temp in x1_label]
    y1_label = axes.get_yticklabels() 
    [y1_label_temp.set_fontname('Times New Roman') for y1_label_temp in y1_label]
    [y1_label_temp.set_fontsize(12) for y1_label_temp in y1_label]

    plt.show()


from matplotlib import cm
from matplotlib.ticker import LinearLocator
from matplotlib import cbook
from matplotlib.colors import LightSource
import matplotlib.animation as animation
from matplotlib import rc
rc('animation', html='jshtml')

def plot_animation_3d(sens, zlim=0.5):
  '''画灵敏度迭代变化图'''
  fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
  # Make data.
  nely, nelx = sens.shape[1], sens.shape[2]
  Y = np.arange(nely)
  X = np.arange(nelx)
  X, Y = np.meshgrid(X, Y)

  surf = [ax.plot_surface(X, Y, sens[0], cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)]
  def update_frames(step, sens, surf):
    surf[0].remove()
    surf[0] = ax.plot_surface(X, Y, sens[step], cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)
    ax.set_ylim(nely,0)
    ax.set_zlim(0, zlim*sens[0].min())
  # Customize the z axis.
  ax.zaxis.set_major_locator(LinearLocator(10))
  # Add a color bar which maps values to colors.
  # fig.colorbar(surf, shrink=0.5, aspect=5)


  ani = animation.FuncAnimation(fig, update_frames, len(sens),fargs=(sens,surf))
  return ani

def plot_animation_2d(sens):
  fig, ax = plt.subplots()
  ims = []
  for i in range(len(sens)):
    im = ax.imshow(sens[i], animated=True, cmap='Greys')
    ims.append([im])
  ani = animation.ArtistAnimation(fig, ims, blit=True,repeat_delay=1000)
  return ani


In [ ]:
def separate_u(u, nelx, nely):
  '''将以行向量形式存储的位移计算结果分成ux、uy'''
  ux, uy = u[::2].reshape(nelx+1, nely+1), u[1::2].reshape(nelx+1, nely+1)
  return ux.T, uy.T

def combine_u(ux, uy):
  '''将ux、uy还原为行向量'''
  return np.dstack((ux.T,uy.T)).ravel()

下面是维度缩减的相关方法：

In [1]:
def scale_reduction_square(problem, factor=2):
  '''对随机生成的方形结构进行维度缩减'''
  # reduce the scale of the problem by factor
  if problem.width % factor != 0 or problem.height % factor != 0:
    raise ValueError(
          f'shape {problem.width} , {problem.height} cannot be reduced by {factor}x')
  width, height = problem.width // factor, problem.height // factor
  normals = np.zeros((width + 1, height + 1, 2))
  forces = np.zeros((width + 1, height + 1, 2))

  non_trivial_normals = np.nonzero(problem.normals)
  non_trivial_forces = np.nonzero(problem.forces)
  # apply boundary conditions to the new problem
  for x_ord, y_ord, direction in zip(non_trivial_normals[0],
                                     non_trivial_normals[1], 
                                     non_trivial_normals[2]):
    if x_ord % factor != 0 or y_ord % factor != 0:
      raise ValueError(
          f'fixed point ({x_ord} , {y_ord}) cannot be reduced accurately by {factor}x')
    normals[x_ord//factor, y_ord//factor, direction] = 1
  for x_ord, y_ord, direction in zip(non_trivial_forces[0],
                                     non_trivial_forces[1], 
                                     non_trivial_forces[2]):
    if x_ord % factor != 0 or y_ord % factor != 0:
      raise ValueError(
          f'force point ({x_ord} , {y_ord}) cannot be reduced accurately by {factor}x')
    forces[x_ord//factor, y_ord//factor, direction] = \
                                    problem.forces[x_ord][y_ord][direction]
  if type(problem.mask) == int:
    return Problem(normals, forces, problem.density)
  
  if not problem.mask.all():
    mask = np.zeros((width, height))
    opt_area = np.nonzero(problem.mask)
    for x_ord, y_ord in zip(opt_area[0], opt_area[1]):
      mask[x_ord//factor, y_ord//factor] = 1
  
  return Problem(normals, forces, problem.density, mask)

def scale_reduction_complex(problem, factor=2):
  '''对problems.py中定义的拓扑优化问题进行维度缩减。
  注意：使用本方法时需要人工或者结合下方scale_reduction_test()函数对结构是否可以直接缩减作判断
  本项目大部分缩减不采用本方法，而是直接调用每一结构特定的problem方法。
  '''
  width, height = problem.width // factor, problem.height // factor
  normals = np.zeros((width + 1, height + 1, 2))
  forces = np.zeros((width + 1, height + 1, 2))
  for x_ord in range(width+1):
    for y_ord in range(height+1):
      for i in range(2):
        normals[x_ord,y_ord,i] = problem.normals[x_ord*factor,y_ord*factor,i]
        forces[x_ord,y_ord,i] = problem.forces[x_ord*factor,y_ord*factor,i]
 
  if type(problem.mask) == int:
    return Problem(normals, forces, problem.density)
  
  if not problem.mask.all():
    mask = np.zeros((width, height))
    opt_area = np.nonzero(problem.mask)
    for x_ord, y_ord in zip(opt_area[0], opt_area[1]):
      mask[x_ord//factor, y_ord//factor] = 1
  
  return Problem(normals, forces, problem.density, mask)


def scale_reduction_test(problem, factor=2):
  '''测试结构维度是否可以被完美缩减（实际项目中未使用）'''
  # 结构mesh维度
  if problem.width % factor != 0 or problem.height % factor != 0:
    raise ValueError(
          f'shape {problem.width} , {problem.height} cannot be reduced by {factor}x')
  # 结构约束的位置
  xords, yords, _ = np.nonzero(problem.normals)
  for xord, yord in zip(xords, yords):
    if xord % factor or yord % factor:
      raise ValueError(
            f'fixed point applied at ({xord},{yord}), cannot be reduced by {factor}x')
  # 结构荷载的位置
  xords, yords, _ = np.nonzero(problem.forces)
  for xord, yord in zip(xords, yords):
    if xord % factor or yord % factor:
      raise ValueError(
            f'fixed point applied at ({xord},{yord}), cannot be reduced by {factor}x')
  return True

def average_downsample(dens, factor=2):
  '''使用平均下采样方法对密度进行维度缩减'''
  if dens.shape[1] % factor != 0 or dens.shape[0] % factor != 0:
    raise ValueError(
          f'shape {dens.shape[0]} , {dens.shape[1]} cannot be reduced by {factor}x')
  b = dens.shape[1] // factor
  return dens.reshape(-1, factor, b, factor).sum((-1, -3)) / (factor*factor)

def bilinear_upsampling(image, factor=2):
  '''双线性插值进行上采样'''
  '''upsample 2d array with shape (nelx, nely)'''
  return np.asarray(tf.image.resize(
      image[:,:,None],[image.shape[0]*factor,image.shape[1]*factor])).squeeze()

def bilinear_upsampling_batch(image, factor=2):
  '''upsample 3d array with shape (batch_size, nelx, nely)'''
  return np.asarray(tf.image.resize(
      image[:,:,:,None],[image.shape[1]*factor,image.shape[2]*factor])).squeeze()

下面是计算结构的单元应力分布特点时涉及的函数：

In [ ]:
def get_stress(disp, nelx, nely, young=1, poisson=0.3):
  '''输入节点位移，计算单元应力、主应力。
  disp为displace()函数计算出的位移向量'''
  ux, uy = separate_u(disp, nelx, nely)
  e, nu = young, poisson
  gaussion_point = [-0.57735, 0.57735]
  c = np.array([[e/(1-nu**2), nu*e/(1-nu**2), 0],
        [nu*e/(1-nu**2), e/(1-nu**2), 0],
        [0, 0, e/(2*(1+nu))]])  
  all_stress, all_diff, all_principle, all_angle = [], [], [], []
  for ux, uy in zip(view_as_windows(ux, window_shape=(2,2), step=(1,1)).reshape(-1,2,2), view_as_windows(uy, window_shape=(2,2), step=(1,1)).reshape(-1,2,2)):
    stress_vector = np.zeros((3,))
    for i in range(2):
      for j in range(2):
        x, y = gaussion_point[i], gaussion_point[j]
        b = np.array([[-(1-y)/4, 0, (1-y)/4, 0, (1+y)/4, 0, -(1+y)/4, 0],
          [0, -(1-x)/4, 0, -(1+x)/4, 0, (1+x)/4, 0, (1-x)/4],
          [-(1-x)/4, -(1-y)/4, -(1+x)/4, (1-y)/4, (1+x)/4, (1+y)/4, (1-x)/4, -(1+y)/4]])
        ue = np.array([ux[0][0], uy[0][0], ux[0][1], uy[0][1], ux[1][1], uy[1][1], ux[1][0], uy[1][0]])
        stress_vector += np.dot(np.dot(c,b),ue)
        sig_x, sig_y, tao_xy = stress_vector[0], stress_vector[1], stress_vector[2]
        sig_max = (sig_x+sig_y)/2 + np.sqrt(((sig_x-sig_y)/2)**2 + tao_xy**2)
        sig_min = (sig_x+sig_y)/2 - np.sqrt(((sig_x-sig_y)/2)**2 + tao_xy**2)
        diff = sig_max - sig_min
        # angle = np.arctan(sig_min/sig_max)
        angle = np.arccos((sig_max+sig_min)/np.sqrt(sig_max**2 + sig_min**2))
    all_principle.append([sig_max, sig_min])
    all_stress.append(stress_vector)
    all_diff.append(diff)
    all_angle.append(angle)
  return np.array(all_stress), np.array(all_diff), np.array(all_principle), np.array(all_angle)

def disp_solver(args, penal=3, e_min=1e-9, e_0=1):
  '''receive problem args, return displacements'''
  x_phys = np.ones((args['nely'], args['nelx'])) * args['mask'] # 将空心区域设为密度极小
  ke = get_stiffness_matrix(args['young'], args['poisson'])
  kwargs = dict(penal=args['penal'], e_min=args['young_min'], e_0=args['young'])
  forces = calculate_forces(x_phys, args)
  u = displace(
      x_phys, ke, forces, args['freedofs'], args['fixdofs'], **kwargs)
  return u

def get_diffs(problem):
  nelx, nely = problem.width, problem.height
  disp = disp_solver(args=specified_task(problem=problem))
  _, all_diff, _, _ = get_stress(disp, nelx, nely, young=1, poisson=0.3)
  # 筛掉mask区域的单元
  if not np.all(problem.mask):
    mask = problem.mask.flatten()
    all_diff = np.delete(all_diff, np.where(mask==0))
  return all_diff

def get_principles(problem):
  nelx, nely = problem.width, problem.height
  disp = disp_solver(args=specified_task(problem=problem))
  _, _, all_principle, _ = get_stress(disp, nelx, nely, young=1, poisson=0.3)
  # 筛掉mask区域的单元
  if not np.all(problem.mask):
    mask = problem.mask.flatten()
    all_principle = np.delete(all_principle, np.where(mask==0), axis=0)
  return all_principle

def get_angles(problem):
  nelx, nely = problem.width, problem.height
  disp = disp_solver(args=specified_task(problem=problem))
  _, _, _, all_angle = get_stress(disp, nelx, nely, young=1, poisson=0.3)
  # 筛掉mask区域的单元
  if not np.all(problem.mask):
    mask = problem.mask.flatten()
    all_angle = np.delete(all_angle, np.where(mask==0), axis=0)
  return all_angle


def plot_histogram(problem, problem_name=None):
  '''画单元应力特征角度直方图'''
  all_angle = get_angles(problem) / np.pi * 180
  # ~is_outlier(all_angle)
  # filtered = all_angle[~is_outlier(all_angle)]
  fig, ax = plt.subplots(figsize=(6,4))
  plt.hist(all_angle, bins=np.arange(181)[::10], weights=[100./len(all_angle)]*len(all_angle), edgecolor="black")
  plt.title(problem_name,fontproperties=simtime,fontsize=15)
  x1_label = ax.get_xticklabels() 
  [x1_label_temp.set_fontname('Times New Roman') for x1_label_temp in x1_label]
  [x1_label_temp.set_fontsize(15) for x1_label_temp in x1_label]
  y1_label = ax.get_yticklabels() 
  [y1_label_temp.set_fontname('Times New Roman') for y1_label_temp in y1_label]
  [y1_label_temp.set_fontsize(15) for y1_label_temp in y1_label]
  ax.set_xlim(xmin = 0, xmax = 180)
  plt.xlabel("$θ$",fontproperties=simtime,fontsize=15)
  plt.ylabel("频数(%)",fontproperties=simtime,fontsize=15)
  plt.show()

In [ ]:
from scipy import ndimage
def sobel_filters(img, kernel_size=3):
    '''施加sobel过滤器（可以用于边界提取）'''
    if kernel_size == 5:
        Kx = np.array([[-5, 4, 0, 4, 5], 
                       [-8, -10, 0, 10, 8], 
                       [-10, -20, 0, 20, 10],
                       [-8, -10, 0, 10, 8], 
                       [-5, 4, 0, 4, 5]
                       ], np.float32)
        Ky = np.array([[5, 8, 10, 8, 5], 
                       [4, 10, 20, 10, 4], 
                       [0, 0, 0, 0, 0],
                       [-4, -10, -20, -10, -4], 
                       [-5, -8, -10, -8, -5] 
                       ], np.float32)
    else:
        Kx = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]], np.float32)
        Ky = np.array([[1, 2, 1], [0, 0, 0], [-1, -2, -1]], np.float32)

    Ix = ndimage.filters.convolve(img, Kx, mode='constant', cval=0)
    Iy = ndimage.filters.convolve(img, Ky, mode='constant', cval=0)
    G = np.hypot(Ix, Iy)
    G = G / G.max()
    gradient = np.arctan2(Iy,Ix)
    tangent = np.arctan2(-Ix,Iy)
    return G

下面是实验性质的、最终被弃用的一些方法。


In [ ]:
def get_difference(full_scale, reduced_scale):
  factor = full_scale.shape[-1]//reduced_scale.shape[-1]
  upsampled = bilinear_upsampling(reduced_scale, factor)
  difference = full_scale - upsampled
  return difference

def get_difference_batch(full_scale, reduced_scale):
  factor = full_scale.shape[-1]//reduced_scale.shape[-1]
  upsampled = bilinear_upsampling_batch(reduced_scale, factor)
  difference = full_scale - upsampled
  return difference


def nearest_upsampling(image, factor=2):
  '''upsample 2d array with shape (nelx, nely)'''
  return np.asarray(tf.image.resize(
      image[:,:,None],[image.shape[0]*factor,image.shape[1]*factor],
      method='nearest')).squeeze()

def nearest_upsampling_batch(image, factor=2):
  '''upsample 2d array with shape (nelx, nely)'''
  return np.asarray(tf.image.resize(
      image[:,:,:,None],[image.shape[1]*factor,image.shape[2]*factor],
      method='nearest')).squeeze()

def get_ratio(full_scale, reduced_scale):
  factor = full_scale.shape[-1]//reduced_scale.shape[-1]
  upsampled = bilinear_upsampling(reduced_scale, factor)
  ratio = full_scale/upsampled
  return ratio

def get_ratio_batch(full_scale, reduced_scale):
  factor = full_scale.shape[-1]//reduced_scale.shape[-1]
  upsampled = bilinear_upsampling_batch(reduced_scale, factor)
  ratio = full_scale/upsampled
  return ratio